In [1]:
import json
import time

from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import Sampler as AerSampler
from qiskit.transpiler import CouplingMap


In [2]:
# Define number of qubits, and the device for using GPU acceleration
# https://qiskit.github.io/qiskit-aer/stubs/qiskit_aer.AerSimulator.html#qiskit_aer.AerSimulator


# Test 1) ghz_amplitude and 2) compare ghz_counts_aerrun with ghz_counts_aersampler

# Statevector accepts only 32 qubits max

# maybe defining a initial mapping will reduce the execution time of the transpiling process

In [3]:
def ghz_amplitudes(n, method ='statevector', device=None):

    # Define method (statevector up to 29 qubits)
    # For ghz, we can use stabilizer up to 10000 qubits
    # Or MPS up to 63 qubits, both do not support GPU methods
    # https://qiskit.github.io/qiskit-aer/tutorials/6_extended_stabilizer_tutorial.html
    backend = AerSimulator(method=method, device=device)
    cm = CouplingMap().from_line(n)

    circuit = QuantumCircuit(n)
    circuit.h(0)
    for i in range(n-1):
        circuit.cx(i, i+1)
    
    circuit.save_amplitudes([0, 2**n-1])

    # Test using a specific transpilation/noise model?
    circuit = transpile(circuit, backend=backend, coupling_map=cm)

    # Execute the circuit on the aer simulator
    result = backend.run(circuit).result()

    return result

In [4]:
def ghz_counts_aerrun(n, method ='statevector', shots=10000, device=None):
   
    backend = AerSimulator(method=method, device=device)
    cm = CouplingMap().from_line(n)

    circuit = QuantumCircuit(n)
    circuit.h(0)
    for i in range(n-1):
        circuit.cx(i, i+1)
    circuit.measure_all(inplace=True, add_bits=True)

    # Test using a specific transpilation/noise model?
    circuit = transpile(circuit, backend=backend, coupling_map=cm)

    # Execute the circuit on the aer simulator
    result = backend.run(circuit, shots = shots).result()

    return result

In [5]:
def ghz_counts_aersampler(n, method ='statevector', shots=10000, device=None):

    # Define method (statevector up to 29 qubits)
    # For ghz, we can use stabilizer up to 10000 qubits, with some error
    # Or MPS up to 63 qubits, both do not support GPU methods
    # https://qiskit.github.io/qiskit-aer/tutorials/6_extended_stabilizer_tutorial.html

    
    sampler = AerSampler(backend_options={"method": method, "device": device})
    cm = CouplingMap().from_line(n)

    circuit = QuantumCircuit(n)
    circuit.h(0)
    for i in range(n-1):
        circuit.cx(i, i+1)
    circuit.measure_all(inplace=True, add_bits=True)

    # Test using a specific transpilation/noise model?
    circuit = transpile(circuit, backend=backend, coupling_map=cm)

    # Execute the circuit on the aer simulator
    result = sampler.run([circuit], shots=shots).result()

    return result.quasi_dists

In [6]:
%%timeit
ghz_amplitudes(n=1)

6.95 ms ± 1.21 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%%timeit
ghz_amplitudes(n=10)

8.82 ms ± 743 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
%%timeit
ghz_amplitudes(n=20)

42.3 ms ± 1.81 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
%%timeit
ghz_amplitudes(n=25)

1.09 s ± 123 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%%timeit
ghz_amplitudes(n=27)

3.46 s ± 67.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%%timeit
ghz_amplitudes(n=29)

15.4 s ± 746 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
# Insufficent memory for circuits over 29 qubits
#%%timeit -r7 -n7
